# ✨ UnQCreator Engine Launcher

Welcome to the UnQCreator Engine Launcher! This notebook sets up your cloud-based engine backend for creating amazing videos.

## 🚀 Quick Start:
1. Click the "Run" button on the cell below to start the engine
2. Wait for the setup to complete (5-10 minutes)
3. Copy the public URL that will be displayed
4. Use this URL in your frontend Settings page

That's it! No configuration needed.

In [ ]:
#@title 🚀 Launch UnQCreator Engine
#@markdown Click the play button to start the UnQCreator Engine. A public URL will be generated for you.

import os
import time
import asyncio
import re
from subprocess import Popen, PIPE
from IPython.display import HTML, display

# Display a stylish progress indicator
def show_progress(message):
    print(f"\033[1;34m{message}\033[0m")
    
# Step 1: Clone the repository
show_progress("📥 Cloning the UnQCreator repository...")
!git clone https://github.com/Sandeepgaddam5432/unqcreator.git /content/unqcreator
%cd /content/unqcreator
show_progress("✅ Repository cloned successfully!")

# Step 2: Install dependencies with correct path to requirements.txt
show_progress("📦 Installing Python dependencies... (This may take a few minutes)")
!pip install -q -r video_genarater/UnQ-YT-Auto/requirements.txt
show_progress("✅ Dependencies installed successfully!")

# Step 3: Setup Cloudflare Quick Tunnel
show_progress("🔒 Setting up Cloudflare Quick Tunnel...")
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /usr/local/bin/cloudflared
!chmod +x /usr/local/bin/cloudflared
show_progress("✅ Cloudflare client downloaded and installed!")

# Step 4: Launch Quick Tunnel and capture the URL
show_progress("🚇 Starting Cloudflare Quick Tunnel...")
# Create a log file to capture the tunnel output
!touch /tmp/cloudflared.log
# Start the tunnel in the background and redirect output to the log file
tunnel_process = Popen(
    ['/usr/local/bin/cloudflared', 'tunnel', '--url', 'http://localhost:8188'],
    stdout=PIPE, stderr=PIPE, universal_newlines=True
)

# Function to extract the tunnel URL from cloudflared output
def get_tunnel_url():
    # Wait for the tunnel to initialize
    time.sleep(5)
    
    # Read from the process output
    tunnel_url = None
    max_attempts = 10
    attempt = 0
    
    while tunnel_url is None and attempt < max_attempts:
        attempt += 1
        # Read from stderr as cloudflared logs to stderr
        line = tunnel_process.stderr.readline()
        if not line:
            time.sleep(1)
            continue
            
        # Look for the trycloudflare.com URL in the output
        match = re.search(r'https://[\w-]+\.trycloudflare\.com', line)
        if match:
            tunnel_url = match.group(0)
            break
    
    # If we couldn't find the URL in the process output, try a different approach
    if tunnel_url is None:
        # Run cloudflared tunnel info command to get the URL
        info_output = !cloudflared tunnel info
        for line in info_output:
            match = re.search(r'https://[\w-]+\.trycloudflare\.com', line)
            if match:
                tunnel_url = match.group(0)
                break
    
    return tunnel_url

# Get the tunnel URL
tunnel_url = get_tunnel_url()

if tunnel_url:
    show_progress(f"✅ Cloudflare Tunnel established!")
    
    # Display final setup message
    show_progress("\n🎬 Starting UnQCreator Engine...")
    
    # Show pretty note box with the tunnel URL
    display(HTML(f"""
    <div style="background-color: #f0f7ff; padding: 15px; border-left: 5px solid #0366d6; border-radius: 5px; margin: 20px 0;">
      <h3 style="margin-top: 0; color: #0366d6;">🎉 Your Engine is Ready!</h3>
      <p>The UnQCreator Engine is now available at:</p>
      <p><code>{tunnel_url}</code></p>
      <p>Copy this URL and use it in your frontend Settings page.</p>
    </div>
    """))
else:
    show_progress("❌ Could not establish Cloudflare Tunnel. Please try again.")

# Launch the server with the correct path and allow any origin
print("\n📡 Engine output will appear below:\n")
print("------------------------------------------")
!cd video_genarater/UnQ-YT-Auto && python main.py --listen --enable-cors-header "*"